# Qiskit Runtime REST API query & job submission

## Query API without authentication

In [25]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/programs'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

[program['id'] for program in data['programs']]

['qasm3-runner', 'sampler', 'estimator', 'circuit-runner']

## Get temporary Access token from Auth API via API Token

In [26]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [28]:
url_backends = 'https://runtime-us-east.quantum-computing.ibm.com/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

print(backends_response.json()['devices'][:5],"...")

['simulator_mps', 'simulator_statevector', 'ibm_perth', 'ibmq_qasm_simulator', 'simulator_stabilizer'] ...


## Run a job

### set up qasm circuit

In [29]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

### run circuit via API

In [48]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/jobs'
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibm_kyiv', #Sessions work best with real backends (QCs)
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "start_session": "True", #set to False if you just need to run a single job. When addressing simulators, comment out this line. 
    "params": {
        "circuits": [qasm_string]*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cn9rrnt0vr7g008m3c40","backend":"ibm_kyiv","session_id":"cn9rrnt0vr7g008m3c40"}


### Run follow-up jobs in the same Session (optional)

In [43]:
session_id=response.json()['session_id']
backend=response.json()['backend']
[session_id,backend]

['cn9q5dxpb7t0008r3wp0', 'ibm_kyiv']

In [45]:
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "session_id": session_id,
    "params": {
        "circuits": [qasm_string]*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cn9q5sq1mh9g008yf980","backend":"ibm_kyiv","session_id":"cn9q5dxpb7t0008r3wp0"}


### Query Websocket for live job update and close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. So we will listen for a status change of the last job via websocket and close the Session when done.

In [49]:
import asyncio
import websockets

with open('token') as file:
    api_key=file.read()

async def receive_status_updates(api_key):
    uri = "wss://runtime-us-east.quantum-computing.ibm.com/stream/jobs/"+job_id 
    extra_headers = {'Authorization': f'Bearer {api_key}'}

    async with websockets.connect(uri, extra_headers=extra_headers) as websocket:
        while True:
            try:
                message = await websocket.recv()
                print(f"Received message: {message}")  
            except websockets.exceptions.ConnectionClosed:
                print("WebSocket connection closed.")
                break

await receive_status_updates(api_key)

#subsequently closing Session
status_result = requests.get(url+'/'+job_id,headers=headers)
sessionsurl = 'https://runtime-us-east.quantum-computing.ibm.com/sessions/'+session_id+'/close'

if status_result.json()['state']['status']=='Completed':
   response = requests.delete(sessionsurl,headers=headers)
else:
   print('waiting for job to complete before closing Session '+session_id)

if response.status_code==204:
    print('Session '+session_id+' was closed')

Received message: Listening to results for job: cn9rrnt0vr7g008m3c40

WebSocket connection closed.
Session cn9q5dxpb7t0008r3wp0 was closed


### Get results

In [47]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'0': 0.003022889032675519, '1': 0.9969771109673246},
  {'0': 0.0002426399223552338, '1': 0.9997573600776448},
  {'0': 0.0004953898414752601, '1': 0.9995046101585248},
  {'0': 0.0004953898414752601, '1': 0.9995046101585248},
  {'0': 0.003528388870915572, '1': 0.9964716111290844}],
 'metadata': [{'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0357391689128672,
   'readout_mitigation_time': 0.042210309067741036},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0357391689128672,
   'readout_mitigation_time': 0.09641426499001682},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0357391689128672,
   'readout_mitigation_time': 0.0003738719969987869},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0357391689128672,
   'readout_mitigation_time': 0.0003721269313246012},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1

## Invalidate Token

In [22]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 


## Test Token Invalidation

This should yield an Authoritzation error (Error 401) once access token is invalidated

In [23]:
rl = 'https://runtime-us-east.quantum-computing.ibm.com/jobs'
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibmq_qasm_simulator',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "params": {
        "circuits": [qasm_string]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(response.text,"\n")
    print(f"Error: {response.status_code}")

{"errors":[{"code":1201,"message":"Invalid credentials.","solution":"Verify your credentials and try again.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]} 

Error: 401
